In [1]:
import pandas as pd
import numpy as np

In [2]:
data_raw = pd.read_csv('raw_data/most_traded_stocks_data.csv', header=[0,1], index_col=0, parse_dates=True)

In [3]:
# Remove all columns where the second level is 'Volume'
columns_to_drop = [col for col in data_raw.columns if col[1] == 'Volume']
data_raw = data_raw.drop(columns=columns_to_drop)

In [ ]:
# Convert VIX to daily volatility
data_raw['^VIX'] = data_raw['^VIX'] / np.sqrt(252) / 100

In [5]:
# New dataframe with features only
data_features = data_raw.copy().drop(columns=[col for col in data_raw.columns if col[1] != ''])

In [ ]:
# Signal trigger (adjustable and may have an impact on model performance, a parameter to fine-tune later if futher investigation is needed)
Trigger_up = 0.005
Trigger_down = -0.005

# Get unique tickers
tickers = data_raw.columns.get_level_values(0).unique()

# Collect all calculations
data = {}
for ticker in tickers:
    # Select the relevant columns for the ticker
    high = data_raw[(ticker, 'High')]
    low = data_raw[(ticker, 'Low')]
    close = data_raw[(ticker, 'Close')]
    
    # Calculate indicators
    stochastic = np.clip((close - low) / (high - low), 0, 1)
    william = np.clip((high - close) / (high - low), 0, 1)
    basic_mid = (high + low + close) / 3
    close_to_high = high / close - 1
    close_to_low = low / close - 1
    day_to_day_return = close.pct_change()
    signal = np.where(day_to_day_return > Trigger_up, 1, np.where(day_to_day_return < Trigger_down, -1, 0))
    
    # Store in dict data
    data[(ticker, 'Stochastic')] = stochastic
    data[(ticker, 'William')] = william
    data[(ticker, 'Basic_Mid')] = basic_mid
    data[(ticker, 'Close_to_High')] = close_to_high
    data[(ticker, 'Close_to_Low')] = close_to_low
    data[(ticker, 'DD_Return')] = day_to_day_return
    
    # Moving averages (various windows for short-term and long-term trends)
    MA_N = [5, 10, 15, 35, 40, 45]
    Trend_N = [50, 100, 150]
    Series_dict = {'Close': close, 'High': high, 'Low': low}

    for serie in Series_dict.keys():
        matrice = Series_dict[serie]
        for N in MA_N:
            data[(ticker, f'{serie}_MA_{N}')] = matrice.rolling(window=N).mean() / matrice
            data[(ticker, f'{serie}_STD_{N}')] = matrice.rolling(window=N).std() / matrice.rolling(window=N).mean()
            data[(ticker, f'{serie}_GROWTH_{N}')] = (close / close.shift(N)) ** (252 / N) - 1

            data[(ticker, f'{serie}_Above_MA_{N}')] = (matrice > data[(ticker, f'{serie}_MA_{N}')]).astype(int)
            data[(ticker, f'{serie}_Above_STD_{N}')] = (matrice > data[(ticker, f'{serie}_STD_{N}')]).astype(int)
            
    
    for serie in Series_dict.keys():
        matrice = Series_dict[serie]
        for N in Trend_N:
            data[(ticker, f'{serie}_TREND_{N}')] = matrice.rolling(window=N).mean() / matrice
            data[(ticker, f'{serie}_STD_{N}')] = matrice.rolling(window=N).std() / matrice.rolling(window=N).mean()
            data[(ticker, f'{serie}_GROWTH_{N}')] = (close / close.shift(N)) ** (252 / N) - 1

            data[(ticker, f'{serie}_Above_TREND_{N}')] = (matrice > data[(ticker, f'{serie}_TREND_{N}')]).astype(int)
            data[(ticker, f'{serie}_Above_STD_{N}')] = (matrice > data[(ticker, f'{serie}_STD_{N}')]).astype(int)

    for N in MA_N:
        # Correlation with VIX on short-term windows
        data[(ticker, f'Close_CORR_{N}')] = data[(ticker, f'Close_STD_{N}')].rolling(window=N).corr(data_raw[('^VIX', 'Close')].rolling(window=N).mean())
        

    # Adding lagged features for basic indicators
    lag_days = [1,2,3,4,5]
    feature_names = ['Stochastic', 'William', 'Basic_Mid', 'Close_to_High', 'Close_to_Low', 'DD_Return']
    for feature in feature_names:
        for lag in lag_days:
            data[(ticker, f'{feature}_lag_{lag}')] = data[(ticker, feature)].shift(lag)

    # Target signal to predict
    data[(ticker, 'Signal')] = signal
    shifted_signal = np.empty_like(signal)
    shifted_signal[:-1] = signal[1:]      # shift by -1 (predict now the next day's dynamic)
    shifted_signal[-1] = 0                # fill the gap with 0

# Create a DataFrame from the created data
new_columns_df = pd.DataFrame(data)

# Concatenate new features to original features
data_features = pd.concat([data_features, new_columns_df], axis=1)

In [7]:
features = data_features.columns.get_level_values(1).unique()
print(f"features available: {features}")

features available: Index(['Stochastic', 'William', 'Basic_Mid', 'Close_to_High', 'Close_to_Low',
       'DD_Return', 'Close_MA_5', 'Close_STD_5', 'Close_GROWTH_5',
       'Close_Above_MA_5',
       ...
       'Close_to_Low_lag_2', 'Close_to_Low_lag_3', 'Close_to_Low_lag_4',
       'Close_to_Low_lag_5', 'DD_Return_lag_1', 'DD_Return_lag_2',
       'DD_Return_lag_3', 'DD_Return_lag_4', 'DD_Return_lag_5', 'Signal'],
      dtype='object', length=178)


In [8]:
data_features.to_csv('processed_data/data_features.csv')